In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#create df
import pandas as pd
import os
import torch
import numpy as np



root_correlation_folder = '/mnt/data/chris/nodropbox/Projects/circuit_pruner/correlations/'
methods = ['actxgrad','force','snip','magnitude','random']


# folders = {'resnet':'/mnt/data/chris/dropbox/Research-Hamblin/Projects/circuit_pruner/method_comparison/resnet18/circuit_activations',
#            'inception':'/mnt/data/chris/dropbox/Research-Hamblin/Projects/circuit_pruner/method_comparison/inception/circuit_activations',
#            'vgg11':'/mnt/data/chris/dropbox/Research-Hamblin/Projects/circuit_pruner/method_comparison/vgg11/circuit_activations',
#            'vgg11_bn':'/mnt/data/chris/dropbox/Research-Hamblin/Projects/circuit_pruner/method_comparison/vgg11_bn/circuit_activations'}


models = ['inception','resnet18','vgg11']

big_list = []
columns = ['model','sparsity','correlation','layer','unit','method']
sparsities = [.9,.8,.7,.6,.5,.4,.3,.2,.1,.05,.01,.005,.001]



for model in models:
    for method in methods:
        folder = root_correlation_folder+model+'/imagenet_2/'+method
        files = os.listdir(folder)
        for file in files:
            data = torch.load(os.path.join(folder,file))
            for i in range(len(sparsities)):
                try:
                    row = [model,sparsities[i],np.abs(data['correlations'][i]),data['layer'],data['unit'],method]
                except:
                    print(file)
                    continue
                big_list.append(row)
    
    
    
    
df = pd.DataFrame(big_list,columns= columns)
    
    





In [3]:
df = df.fillna(0)
df

,model,sparsity,correlation,layer,unit,method
0,inception,0.900,0.999742,mixed4b_1x1_pre_relu_conv,4,actxgrad
1,inception,0.800,0.996374,mixed4b_1x1_pre_relu_conv,4,actxgrad
2,inception,0.700,0.983742,mixed4b_1x1_pre_relu_conv,4,actxgrad
3,inception,0.600,0.945100,mixed4b_1x1_pre_relu_conv,4,actxgrad
4,inception,0.500,0.877607,mixed4b_1x1_pre_relu_conv,4,actxgrad
...,...,...,...,...,...,...
26736,vgg11,0.100,0.294546,features.6,7,random
26737,vgg11,0.050,0.035036,features.6,7,random
26738,vgg11,0.010,0.000000,features.6,7,random
26739,vgg11,0.005,0.000000,features.6,7,random


In [4]:
import pickle

alexnet_data = '/mnt/data/chris/dropbox/Research-Hamblin/Projects/circuit_pruner/method_comparison/circuit_with_force_and_mag_df.pkl'


alex_df = pickle.load(open(alexnet_data,'rb'))

/home/chris/miniconda3/envs/circuit_pruner/lib/python3.7/site-packages/torch/cuda/__init__.py:125: UserWarning: 
Tesla K40c with CUDA capability sm_35 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_61 sm_70 sm_75 compute_37.
If you want to use the Tesla K40c GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


In [5]:
alex_df_sel = alex_df[["model", "sparsity",'masked_pearson','layer','unit','method']]

alex_df_sel.rename(columns={"masked_pearson": "correlation"})




,model,sparsity,correlation,layer,unit,method
0,alexnet,0.001,0.000000,features_10,0,actxgrad
1,alexnet,0.005,-0.046477,features_10,0,actxgrad
2,alexnet,0.010,0.148798,features_10,0,actxgrad
3,alexnet,0.050,0.086445,features_10,0,actxgrad
4,alexnet,0.100,0.071412,features_10,0,actxgrad
...,...,...,...,...,...,...
1558,alexnet_sparse,0.500,0.992880,features_8,9,magnitude
1559,alexnet_sparse,0.600,0.999977,features_8,9,magnitude
1560,alexnet_sparse,0.700,0.999999,features_8,9,magnitude
1561,alexnet_sparse,0.800,1.000000,features_8,9,magnitude


In [12]:
alex_df_sel

,model,sparsity,masked_pearson,layer,unit,method
0,alexnet,0.001,0.000000,features_10,0,actxgrad
1,alexnet,0.005,-0.046477,features_10,0,actxgrad
2,alexnet,0.010,0.148798,features_10,0,actxgrad
3,alexnet,0.050,0.086445,features_10,0,actxgrad
4,alexnet,0.100,0.071412,features_10,0,actxgrad
...,...,...,...,...,...,...
1558,alexnet_sparse,0.500,0.992880,features_8,9,magnitude
1559,alexnet_sparse,0.600,0.999977,features_8,9,magnitude
1560,alexnet_sparse,0.700,0.999999,features_8,9,magnitude
1561,alexnet_sparse,0.800,1.000000,features_8,9,magnitude


### FULL DISTRIBUTION

In [25]:
import plotly.express as px
import plotly.graph_objs as go

fig = go.Figure()


fig = px.box(df, x="sparsity", y='correlation', color="model")

    
fig.update_xaxes(autorange="reversed",title="Sparsity") 
fig.update_yaxes(title="|Pearson R|")

fig.update_layout({ 'showlegend':True,
                    'width':1200,
                    'plot_bgcolor':'rgba(255,255,255,1)',
                    'paper_bgcolor':'rgba(255,255,255,1)'
                    })
   
fig.update_layout(
    xaxis = dict(
        tickmode = 'array'
    )
)
fig.update_layout(xaxis_type='category')
                
                
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.4,
    xanchor="left",
    x=0.05
))

fig.show()

In [12]:
#plot method diff average by sparsity

import plotly.graph_objects as go
import plotly.express as px
import numpy as np


fig = go.Figure()

model = 'inception'
color_col = 'method'

method_colors = {'actxgrad':px.colors.qualitative.T10[1],
                'snip':px.colors.qualitative.T10[3],
                'force':px.colors.qualitative.T10[4],
                'magnitude':px.colors.qualitative.T10[6],
                'random':px.colors.qualitative.T10[7]
               }


df_plot = df.loc[(df['model'] == model)]

sparsities =  list(df_plot['sparsity'].unique())
sparsities.sort()



models =  list(df_plot['model'].unique())
methods = list(df_plot['method'].unique())



for method in methods:
    x = sparsities
    y = []
    for sparsity in sparsities:
        df_sel = df_plot.loc[(df_plot['sparsity'] == sparsity) & (df_plot['method'] == method) & (df_plot['model'] == model)]
        y.append(np.median(df_sel['correlation']))

    fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    #name=model+':'+method,
                    name = method,
                    line=dict(color=method_colors[method],width=4)
                        )
                    )

fig.update_xaxes(autorange="reversed",title = 'sparsity',gridcolor='rgb(210,210,210)')
fig.update_yaxes(title = '|Pearson R|',gridcolor='rgb(210,210,210)')

fig.update_layout({ 'showlegend':False,
                    'width':600,
                    'plot_bgcolor':'rgba(255,255,255,1)',
                    'paper_bgcolor':'rgba(255,255,255,1)'
                    })
   
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = [.9, .7, .5, .3, .1, .01]
    )
)
     

                
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.4,
    xanchor="left",
    x=0.1),
    font=dict(size=18),
    title = '   '+model
    )
    
fig.show()

In [13]:
fig.write_image('../plots/inception_method_compare.png')

In [39]:

fig = go.Figure()

model = 'resnet18'
color_col = 'method'

method_colors = {'actxgrad':px.colors.qualitative.T10[1],
                'snip':px.colors.qualitative.T10[3],
                'force':px.colors.qualitative.T10[4],
                'magnitude':px.colors.qualitative.T10[6],
                'random':px.colors.qualitative.T10[7]
               }


df_plot = df.loc[(df['model'] == model)]

sparsities =  list(df_plot['sparsity'].unique())
sparsities.sort()



models =  list(df_plot['model'].unique())
methods = list(df_plot['method'].unique())



for method in methods:
    x = sparsities
    y = []
    for sparsity in sparsities:
        df_sel = df_plot.loc[(df_plot['sparsity'] == sparsity) & (df_plot['method'] == method) & (df_plot['model'] == model)]
        y.append(np.median(df_sel['correlation']))

    fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    name=model+':'+method,
                    line=dict(color=method_colors[method],width=4)
                        )
                    )

fig.update_xaxes(autorange="reversed",title = 'sparsity',gridcolor='rgb(210,210,210)')
fig.update_yaxes(title = '|Pearson R|',gridcolor='rgb(210,210,210)')

fig.update_layout({ 'showlegend':True,
                    'width':900,
                    'plot_bgcolor':'rgba(255,255,255,1)',
                    'paper_bgcolor':'rgba(255,255,255,1)'
                    })
   
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = [.9, .7, .5, .3, .1, .01]
    )
)
     

                
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.7,
    xanchor="left",
    x=0.1
))
    
fig.show()

In [40]:
fig = go.Figure()

model = 'inception'
color_col = 'method'

method_colors = {'actxgrad':px.colors.qualitative.T10[1],
                'snip':px.colors.qualitative.T10[3],
                'force':px.colors.qualitative.T10[4],
                'magnitude':px.colors.qualitative.T10[6],
                'random':px.colors.qualitative.T10[7]
               }


df_plot = df.loc[(df['model'] == model)]

sparsities =  list(df_plot['sparsity'].unique())
sparsities.sort()



models =  list(df_plot['model'].unique())
methods = list(df_plot['method'].unique())



for method in methods:
    x = sparsities
    y = []
    for sparsity in sparsities:
        df_sel = df_plot.loc[(df_plot['sparsity'] == sparsity) & (df_plot['method'] == method) & (df_plot['model'] == model)]
        y.append(np.median(df_sel['correlation']))

    fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines',
                    name=model+':'+method,
                    line=dict(color=method_colors[method],width=4)
                        )
                    )

fig.update_xaxes(autorange="reversed",title = 'sparsity',gridcolor='rgb(210,210,210)')
fig.update_yaxes(title = '|Pearson R|',gridcolor='rgb(210,210,210)')

fig.update_layout({ 'showlegend':True,
                    'width':900,
                    'plot_bgcolor':'rgba(255,255,255,1)',
                    'paper_bgcolor':'rgba(255,255,255,1)'
                    })
   
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = [.9, .7, .5, .3, .1, .01]
    )
)
     

                
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.7,
    xanchor="left",
    x=0.1
))
    
fig.show()